In [1]:
import pandas as pd
from openpyxl import Workbook
from datetime import datetime
import openpyxl as opx
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
import json

In [2]:
import sys
sys.path.append(r'/home/nkrokhmal/Desktop/MyProjects/SergeyProject/umalat-project/prod')
from app.models import *
from app.models import Boiling as BoilingModel
from app.models import Termizator as TermizatorModel
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string


from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"/home/nkrokhmal/Desktop/umalat/data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

def cast_sku(obj):
    if isinstance(obj, SKU):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(SKU).filter(SKU.id == obj).first()
    else:
        raise Exception('Unknown sku type')
        
def cast_boiling(obj):
    if isinstance(obj, Boiling):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(Boiling).filter(Boiling.id == obj).first()
    else:
        raise Exception('Unknown boiling type')

In [3]:
skus = session.query(SKU).all()

In [13]:
df = pd.read_excel(r'request_new.xlsx', index_col=0)
df.columns = range(df.shape[1])

In [14]:
indeces = df.loc['Дата выработки продукции:'].dropna().index
df = df[indeces]

In [15]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,128,131
Отчет от,,,,,,,,,,,,,,,,,,,,,
Группа,Кавказский,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Черкесский,NaN,...,NaN,NaN,NaN,NaN,NaN,Некондиционная продукция,Полуфабрикаты,NaN,NaN,Группа
Подгруппа,Кавказский,Кавказский,Кавказский,Кавказский,Кавказский,Кавказский,Кавказский,Кавказский,Черкесский,Черкесский,...,Масло,Масло,Масло,Масло,Масло,Некондиционная продукция,Полуфабрикаты,NaN,NaN,Подгруппа
Бренд,Умалат,Умалат,Умалат,Умалат,Умалат,Глобус,Красная птица,Умалат,Умалат,Умалат,...,Unagrande,Unagrande,Unagrande,Без бренда,Без бренда,NaN,NaN,NaN,NaN,Бренд
Дата выработки продукции:,"Кавказский ""Умалат"" (Перекресток), 45%, кг, в/у","Кавказский ""Умалат"" (Окей), 45%, кг, в/у","Кавказский ""Умалат"" (Тандер), 45%, кг , в/у","Кавказский ""Умалат"" (Метро), 45%, кг, в/у","Кавказский ""Умалат"", 45%, 0,37 кг, в/у","Кавказский ""Глобус"", 45%, кг, в/у","Кавказский ""Красная птица"", 45%, 0,37 кг, в/у","Кавказский ""Умалат"", 45%, кг, в/у","Сыр Черкесский ""Умалат"", 45%, 0,28 кг, т/ф","Сыр Черкесский ""Умалат"", 45%, кг, т/ф, ВЕС",...,"Масло сладко-сливочное соленое Крестьянское ""U...","Масло сладко-сливочное Крестьянское ""Unagrande...",Масло сладко-сливочное без лактозы Крестьянско...,"Масло сладко-сливочное традиционное 84%, 2 кг,...","Масло сладко-сливочное Традиционное, 82,5%, 2 ...",Некондиционная продукция,п/ф Рикотта промышленная,"П/ф Качокавалло 45%, кг",П/ф Качокавалло 45% большие головки,Дата выработки продукции:
Код номенклатуры в 1C,Н0000086159,Н0000086487,Н0000086542,Н0000088745,Н0000084595,Н0000090511,Н0000096641,Н0000080826,Н0000094227,Н0000094228,...,Н0000096291,Н0000096292,Н0000096293,Н0000093768,Н0000088626,NaN,Н0000079224,Н0000083041,Н0000092414,Код номенклатуры в 1C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,"Кавказский ""Умалат"" (Окей), 45%, кг, в/у","Кавказский ""Умалат"" (Перекресток), 45%, кг, в/у","Кавказский ""Умалат"" (половинки), 45%, кг, в/у","Кавказский ""Умалат"" (Тандер), 45%, кг, в/у","Кавказский ""Умалат"", 45%, 0,37 кг, в/у",NaN,NaN,"Качокавалло ""Ungrande"", 45%, 0,26 кг, в/у",NaN,NaN,...,NaN,NaN,NaN,NaN,Сырная крошка,"Четук ""Умалат"", 45%, 0,37 кг, в/у","Чечил ""Умалат"", 43%, 0,19 кг, т/ф",NaN,NaN,NaN
Складе ГП,6,70.7,7.768,82.017,344.1,NaN,NaN,849.68,NaN,NaN,...,NaN,NaN,NaN,NaN,70.4,136.6,121.41,NaN,NaN,-16155.2


In [16]:
tmp = df.loc[['Дата выработки продукции:', 'Заявлено всего, кг:', 'Фактические остатки на складах - Заявлено, кг:', 'Нормативные остатки, кг']].fillna(0)

In [17]:
tmp

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,128,131
Отчет от,,,,,,,,,,,,,,,,,,,,,
Дата выработки продукции:,"Кавказский ""Умалат"" (Перекресток), 45%, кг, в/у","Кавказский ""Умалат"" (Окей), 45%, кг, в/у","Кавказский ""Умалат"" (Тандер), 45%, кг , в/у","Кавказский ""Умалат"" (Метро), 45%, кг, в/у","Кавказский ""Умалат"", 45%, 0,37 кг, в/у","Кавказский ""Глобус"", 45%, кг, в/у","Кавказский ""Красная птица"", 45%, 0,37 кг, в/у","Кавказский ""Умалат"", 45%, кг, в/у","Сыр Черкесский ""Умалат"", 45%, 0,28 кг, т/ф","Сыр Черкесский ""Умалат"", 45%, кг, т/ф, ВЕС",...,"Масло сладко-сливочное соленое Крестьянское ""U...","Масло сладко-сливочное Крестьянское ""Unagrande...",Масло сладко-сливочное без лактозы Крестьянско...,"Масло сладко-сливочное традиционное 84%, 2 кг,...","Масло сладко-сливочное Традиционное, 82,5%, 2 ...",Некондиционная продукция,п/ф Рикотта промышленная,"П/ф Качокавалло 45%, кг",П/ф Качокавалло 45% большие головки,Дата выработки продукции:
"Заявлено всего, кг:",237.6,144,1065,195,1279.83,150,0,33,1258.88,4.64,...,135,61,18,240,768,0,0,0,0,"Заявлено всего, кг:"
"Фактические остатки на складах - Заявлено, кг:",10.406,9.708,-518.094,4.618,-285.27,0,0,0.272,-663.04,3.36,...,201,245,351,186,-24,0,0,0,0,"Фактические остатки на складах - Заявлено, кг:"
"Нормативные остатки, кг",957.586,163.119,661.891,61.8281,856.585,29.1429,0,201.208,661.2,90.0394,...,0,0,0,259.048,472.19,0,0,0,0,"Нормативные остатки, кг"


In [18]:
tmp.iloc[:, :-1]


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,128
Отчет от,,,,,,,,,,,,,,,,,,,,,
Дата выработки продукции:,"Кавказский ""Умалат"" (Перекресток), 45%, кг, в/у","Кавказский ""Умалат"" (Окей), 45%, кг, в/у","Кавказский ""Умалат"" (Тандер), 45%, кг , в/у","Кавказский ""Умалат"" (Метро), 45%, кг, в/у","Кавказский ""Умалат"", 45%, 0,37 кг, в/у","Кавказский ""Глобус"", 45%, кг, в/у","Кавказский ""Красная птица"", 45%, 0,37 кг, в/у","Кавказский ""Умалат"", 45%, кг, в/у","Сыр Черкесский ""Умалат"", 45%, 0,28 кг, т/ф","Сыр Черкесский ""Умалат"", 45%, кг, т/ф, ВЕС",...,"Масло сливочное ""Умалат"", 72,5%, 2 кг, к/к","Масло сладко-сливочное соленое Крестьянское ""U...","Масло сладко-сливочное Крестьянское ""Unagrande...",Масло сладко-сливочное без лактозы Крестьянско...,"Масло сладко-сливочное традиционное 84%, 2 кг,...","Масло сладко-сливочное Традиционное, 82,5%, 2 ...",Некондиционная продукция,п/ф Рикотта промышленная,"П/ф Качокавалло 45%, кг",П/ф Качокавалло 45% большие головки
"Заявлено всего, кг:",237.6,144,1065,195,1279.83,150,0,33,1258.88,4.64,...,762,135,61,18,240,768,0,0,0,0
"Фактические остатки на складах - Заявлено, кг:",10.406,9.708,-518.094,4.618,-285.27,0,0,0.272,-663.04,3.36,...,702,201,245,351,186,-24,0,0,0,0
"Нормативные остатки, кг",957.586,163.119,661.891,61.8281,856.585,29.1429,0,201.208,661.2,90.0394,...,696.762,0,0,0,259.048,472.19,0,0,0,0


In [19]:
tmp = df.dropna(thresh=limit-2)
tmp.dropna(how='all', axis='columns')

NameError: name 'limit' is not defined

In [20]:
sku_list = tmp.iloc[0].dropna()

In [21]:
sku_name = [x for x in list(sku_list) if 'Контрагент' not in x and 'Итог' not in x]

In [35]:
sku_name = list(sku_list[1:-1])

In [37]:
sku_volumes = list(tmp.iloc[-1].dropna()[1:-1])

In [39]:
list(zip(sku_name, sku_volumes))

[('Кавказский "Глобус", 45%, кг, в/у, кг', 42),
 ('Кавказский "Глобус", 45%, кг, в/у, кг', 42),
 ('Кавказский "Умалат" (Метро), 45%, кг, в/у, кг', 81),
 ('Кавказский "Умалат" (Метро), 45%, кг, в/у, кг', 81),
 ('Кавказский "Умалат" (Окей), 45%, кг, в/у, кг', 225),
 ('Кавказский "Умалат" (Окей), 45%, кг, в/у, кг', 225),
 ('Кавказский "Умалат" (Перекресток), 45%, кг, в/у, кг', 260.7),
 ('Кавказский "Умалат" (Перекресток), 45%, кг, в/у, кг', 260.7),
 ('Кавказский "Умалат" (Тандер), 45%, кг , в/у , кг', 708),
 ('Кавказский "Умалат" (Тандер), 45%, кг , в/у , кг', 708),
 ('Кавказский "Умалат", 45%, 0,37 кг, в/у, кг', 740.74),
 ('Кавказский "Умалат", 45%, 0,37 кг, в/у, кг', 740.74),
 ('Кавказский "Умалат", 45%, кг, в/у, кг', 6),
 ('Кавказский "Умалат", 45%, кг, в/у, кг', 6),
 ('Качокавалло "Unagrande" (ОК), 45%, кг, кг', 17.1),
 ('Качокавалло "Unagrande" (ОК), 45%, кг, кг', 17.1),
 ('Качокавалло "Unagrande", 45%, 0,26 кг, в/у, (8 шт), кг', 214.24),
 ('Качокавалло "Unagrande", 45%, 0,26 кг, в/у

In [32]:
list(zip(*tmp.values.tolist()))

[('Кавказский "Умалат" (Перекресток), 45%, кг, в/у', 1098.6, 1455),
 ('Кавказский "Умалат" (Окей), 45%, кг, в/у', -225, 0),
 ('Кавказский "Умалат" (Тандер), 45%, кг , в/у', 213, 972),
 ('Кавказский "Умалат" (Метро), 45%, кг, в/у', 0, 102),
 ('Кавказский "Умалат", 45%, 0,37 кг, в/у', -77.70000000000005, 1252.08),
 ('Кавказский "Глобус", 45%, кг, в/у', -42, 0),
 ('Кавказский "Умалат", 45%, кг, в/у', -16, 12),
 ('Сыр Черкесский "Умалат", 45%, 0,28 кг, т/ф', -20.159999999999968, 712.32),
 ('Сыр Черкесский "Умалат", 45%, кг, т/ф, ВЕС', -3.759999999999998, 49.28),
 ('Сыр Черкесский "Умалат" (БИЛЛА), 45%, т/ф, ВЕС', 0, 0),
 ('Сыр Черкесский "Умалат" (ДИКСИ), 45%, т/ф, ВЕС', 0, 0),
 ('Свели-Квели "Умалат", 30%, 0,37 кг, в/у', -106.55999999999999, 26.64),
 ('Четук "Умалат", 45%, 0,37 кг, в/у', -172.42000000000002, 26.64),
 ('Качорикотта "Unagrande", 45%, 0,37 кг, в/у', -74, 14.8),
 ('Рикотта "Pretto" (зернистая), 30%, 0,37 кг, в/у',
  -150.95999999999998,
  23.68),
 ('Сулугуни "Маркет Перекрест

In [23]:
sku_name

['Кавказский "Умалат" (Перекресток), 45%, кг, в/у',
 'Кавказский "Умалат" (Окей), 45%, кг, в/у',
 'Кавказский "Умалат" (Тандер), 45%, кг , в/у',
 'Кавказский "Умалат" (Метро), 45%, кг, в/у',
 'Кавказский "Умалат", 45%, 0,37 кг, в/у',
 'Кавказский "Глобус", 45%, кг, в/у',
 'Кавказский "Красная птица", 45%, 0,37 кг, в/у',
 'Кавказский "Умалат", 45%, кг, в/у',
 'Сыр Черкесский "Умалат", 45%, 0,28 кг, т/ф',
 'Сыр Черкесский "Умалат", 45%, кг, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (БИЛЛА), 45%, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (ДИКСИ), 45%, т/ф, ВЕС',
 'Свели-Квели "Умалат", 30%, 0,37 кг, в/у',
 'Четук "Умалат", 45%, 0,37 кг, в/у',
 'Качорикотта "Unagrande", 45%, 0,37 кг, в/у',
 'Рикотта "Pretto" (зернистая), 30%, 0,37 кг, в/у',
 'Сулугуни "Маркет Перекресток", 45%, 0,28 кг, т/ф',
 'Сулугуни "ВкусВилл", 45%, 0,28 кг, т/ф',
 'Сулугуни "Умалат", 45%, 0,28 кг, т/ф, (8 шт)',
 'Сулугуни "Умалат", 45%, 0,2 кг, т/ф, (9 шт)',
 'Сулугуни палочки "Умалат", 45%, 0,12 кг, т/ф (10 шт.)',
 'Сулугуни "

In [31]:
sku_db_names = [x.name for x in skus]

In [32]:
extra_names = [x for x in sku_name if x not in sku_db_names]

In [35]:
extra_names[:-1]

['Кавказский "Умалат" (Перекресток), 45%, кг, в/у',
 'Кавказский "Умалат" (Окей), 45%, кг, в/у',
 'Кавказский "Умалат" (Тандер), 45%, кг , в/у',
 'Кавказский "Умалат" (Метро), 45%, кг, в/у',
 'Кавказский "Умалат", 45%, 0,37 кг, в/у',
 'Кавказский "Глобус", 45%, кг, в/у',
 'Кавказский "Красная птица", 45%, 0,37 кг, в/у',
 'Кавказский "Умалат", 45%, кг, в/у',
 'Сыр Черкесский "Умалат", 45%, 0,28 кг, т/ф',
 'Сыр Черкесский "Умалат", 45%, кг, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (БИЛЛА), 45%, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (ДИКСИ), 45%, т/ф, ВЕС',
 'Свели-Квели "Умалат", 30%, 0,37 кг, в/у',
 'Четук "Умалат", 45%, 0,37 кг, в/у',
 'Качорикотта "Unagrande", 45%, 0,37 кг, в/у',
 'Рикотта "Pretto" (зернистая), 30%, 0,37 кг, в/у',
 'Сулугуни палочки "Красная птица", 45%, 0,12 кг, т/ф',
 'Сулугуни без лактозы "ВкусВилл", 45%, 0,2 кг, т/ф',
 'Качокавалло "Unagrande", 45%, 0,26 кг, в/у, (8 шт)',
 'Качокавалло "Unagrande", 45%, кг',
 'Качокавалло "Unagrande", 45%, кг Х5',
 'Качокавалло "Unagra

In [43]:
data = ['Кавказский "Умалат" (Перекресток), 45%, кг, в/у',
 'Кавказский "Умалат" (Окей), 45%, кг, в/у',
 'Кавказский "Умалат" (Тандер), 45%, кг , в/у',
 'Кавказский "Умалат" (Метро), 45%, кг, в/у',
 'Кавказский "Умалат", 45%, 0,37 кг, в/у',
 'Кавказский "Глобус", 45%, кг, в/у',
 'Кавказский "Красная птица", 45%, 0,37 кг, в/у',
 'Кавказский "Умалат", 45%, кг, в/у',
 'Сыр Черкесский "Умалат", 45%, 0,28 кг, т/ф',
 'Сыр Черкесский "Умалат", 45%, кг, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (БИЛЛА), 45%, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (ДИКСИ), 45%, т/ф, ВЕС',
 'Свели-Квели "Умалат", 30%, 0,37 кг, в/у',
 'Четук "Умалат", 45%, 0,37 кг, в/у',
 'Качорикотта "Unagrande", 45%, 0,37 кг, в/у',
 'Рикотта "Pretto" (зернистая), 30%, 0,37 кг, в/у',
 'Рикотта "Unagrande", 50%, 0,25 кг, пл/с',
 'Рикотта "Unagrande", 50%, 0,5 кг, пл/с',
 'Рикотта "Aventino", 45%, 0,2 кг, п/с',
 'Рикотта "Unagrande Professionale", 45%, 0,5 кг, пл/с',
 'Рикотта с шоколадом "Unagrande", 30%, 0,18 кг, пл/с',
 'Рикотта с шоколадом "Unagrande", 30%, 0,14 кг, пл/с',
 'Рикотта сицилийская "Unagrande", 55%, 0,3 кг, пл/с',
 'Рикотта сицилийская "Unagrande", 55%, ВЕС, пл/с',
 'Рикотта с шоколадом "ВкусВилл", 30%, 0,14 кг, пл/с',
 'Рикотта "Pretto", 45%, 0,5 кг, пл/с',
 'Рикотта "Pretto", 45%, 0,2 кг, пл/с',
 'Рикотта "Фермерская коллекция", 45%, 0,2 кг, пл/с',
 'Рикотта "ВкусВилл", 45%, 0,18 кг, пл/с (6 шт)',
 'Рикотта "Красная птица", 30%,  0,25 кг, пл/с (6 шт)',
 'Рикотта с ванилью "Красная птица", 30%, 0,2 кг, пл/с',
 'Рикотта шоколадно-ореховая "Красная птица", 35%, 0,2 кг, пл/с',
 'Рикотта "SPAR", 30%, 0,2 кг, пл/с (6 шт)',
 'Рикотта с ванилью "Бонджорно", 30%, 0,2 кг, пл/с',
 'Рикотта с шоколадом "Бонджорно", 30%, 0,2 кг, пл/с',
 'Рикотта шоколадно-ореховая "Бонджорно", 35%, 0,2 кг, пл/с',
 'Рикотта "Глобус", 45%, 0,25 кг, пл/с',
 'Сливки Panna Fresca "Unagrande", 38%, 0,5 л, пл/с',
 'Сливки "Красная птица", 38%, 0,25 л, пл/с',
 'Кремчиз "Unagrande", 70%, 0,5 кг, пл/с',
 'Кремчиз "Красная птица", 75%, 0,2 кг, пл/с',
 'Кремчиз "Pretto", 75%, 0,2 кг, пл/с',
 'Кремчиз "ВкусВилл", 70%, 0,18 кг, пл/с (6шт)',
 'Кремчиз "Фермерская коллекция", 75%, 0,2 кг, пл/с',
 'Кремчиз "Unagrande", 70%, 0,18 кг, пл/с',
 'Творожный "Фермерская коллекция", 65%,0,18 кг,пл/с',
 'Творожный "Pretto", 65%, 0,18 кг, пл/с',
 'Робиола "Unagrande", 65%, 0,18 кг, пл/с',
 'Маскарпоне "Unagrande", 80%, 0,25 кг, пл/с',
 'Маскарпоне "Pretto", 80%, 0,25 кг, пл/с',
 'Маскарпоне "Pretto", 80%, 0,5 кг, пл/с',
 'Маскарпоне "Красная птица", 80%,  0,25 кг, пл/с, (6 шт)',
 'Маскарпоне с шоколадом "Красная птица", 50%, 0,2 кг, пл/с',
 'Маскарпоне "ВкусВилл", 80%, 0,25 кг, пл/с (6 шт)',
 'Маскарпоне "Глобус", 80%, 0,25 кг, пл/с',
 'Маскарпоне "Unаgrande", 80%, 0,5 кг, пл/с',
 'Маскарпоне с шоколадом "Бонджорно", 50%, 0,2 кг, пл/с',
 'Маскарпоне с шоколадом "Unagrande", 50%, 0,18 кг, пл/с',
 'Масло сливочное "Умалат", 72,5%  0,5 кг, к/к',
 'Масло сливочное "Умалат", 72,5%, 2 кг, к/к',
 'Масло сладко-сливочное соленое Крестьянское "Unagrande", 72,5%, 0,5 кг, к/к',
 'Масло сладко-сливочное Крестьянское "Unagrande", 72,5%, 0,5 кг, к/к',
 'Масло сладко-сливочное без лактозы Крестьянское "Unagrande", 72,5%, 0,5 кг, к/к',
 'Масло сладко-сливочное традиционное 84%, 2 кг, кор (3 вложения)',
 'Масло сладко-сливочное Традиционное, 82,5%, 2 кг, к/к',
 'Некондиционная продукция',
 'п/ф Рикотта промышленная']

In [44]:
with open('data.json', 'w') as jsonfile:
    json.dump(data, jsonfile, ensure_ascii=False)

In [48]:
with open('ignore_sku.json') as json_file:
    data = json.load(json_file)

In [4]:
skus

[<SKU 1>,
 <SKU 2>,
 <SKU 3>,
 <SKU 4>,
 <SKU 5>,
 <SKU 6>,
 <SKU 7>,
 <SKU 8>,
 <SKU 9>,
 <SKU 10>,
 <SKU 11>,
 <SKU 12>,
 <SKU 13>,
 <SKU 14>,
 <SKU 15>,
 <SKU 16>,
 <SKU 17>,
 <SKU 18>,
 <SKU 19>,
 <SKU 20>,
 <SKU 21>,
 <SKU 22>,
 <SKU 23>,
 <SKU 24>,
 <SKU 25>,
 <SKU 26>,
 <SKU 27>,
 <SKU 28>,
 <SKU 29>,
 <SKU 30>,
 <SKU 31>,
 <SKU 32>,
 <SKU 33>,
 <SKU 34>,
 <SKU 35>,
 <SKU 36>,
 <SKU 37>,
 <SKU 38>,
 <SKU 39>,
 <SKU 40>,
 <SKU 41>,
 <SKU 45>,
 <SKU 46>,
 <SKU 47>,
 <SKU 48>,
 <SKU 49>,
 <SKU 50>,
 <SKU 51>,
 <SKU 53>,
 <SKU 54>,
 <SKU 55>,
 <SKU 56>,
 <SKU 57>,
 <SKU 58>,
 <SKU 59>,
 <SKU 60>,
 <SKU 61>,
 <SKU 62>]

In [38]:
def f(sku):
    return {
        'Название SKU': sku.name,
        'Процент': sku.boilings[0].percent,
        'Наличие лактозы': 'Да' if sku.boilings[0].is_lactose else 'Нет',
        'Тип закваски': sku.boilings[0].ferment,
        'Название форм фактора': sku.form_factor.name,
        'Линия': 'Соль' if sku.boilings[0].boiling_type == 'salt' else 'Вода',
        'Имя бренда': sku.brand_name,
        'Вес нетто': sku.weight_netto,
        'Вес форм фактора': sku.boiling_form_factors[0].weight,
        'Выход': sku.output_per_ton,
        'Срок хранения': sku.shelf_life,
        'Является ли SKU теркой': 'Да' if sku.is_rubber else 'Нет',
        'Упаковщик': sku.packer.name,
        'Тип упаковки': sku.pack_types.name if sku.pack_types else None,
        'Скорость упаковки': sku.packing_speed,
        'Охлаждение 1(для воды)': sku.boiling_form_factors[0].first_cooling_time,
        'Охлаждение 2(для воды)': sku.boiling_form_factors[0].second_cooling_time,
        'Время посолки': sku.boiling_form_factors[0].salting_time,
        'Время налива': sku.boilings[0].pourings.pouring_time,
        'Время отвердевания': sku.boilings[0].pourings.soldification_time,
        'Время нарезки': sku.boilings[0].pourings.cutting_time,
        'Время слива': sku.boilings[0].pourings.pouring_off_time,
        'Дополнительное время': sku.boilings[0].pourings.extra_time        
}

In [39]:
df = pd.DataFrame([f(sku) for sku in skus])

In [40]:
df.to_excel("Все параметры.xlsx")

In [43]:
tmp = pd.read_excel('params.xlsx', index_col=0)

In [44]:
tmp

,Название SKU,Процент,Наличие лактозы,Тип закваски,Название форм фактора,Линия,Имя бренда,Вес нетто,Вес форм фактора,Выход,...,Тип упаковки,Скорость упаковки,Охлаждение 1(для воды),Охлаждение 2(для воды),Время посолки,Время налива,Время отвердевания,Время нарезки,Время слива,Дополнительное время
0,"Моцарелла ""Pretto"" (для бутербродов), 45%, 0,2...",2.7,Да,Сакко,Для пиццы,Соль,Pretto,200,200,850,...,NaN,1020.0,NaN,NaN,90.0,50,30,20,15,10
1,"Моцарелла ""Pretto"", 45%, 1,2 кг, в/у",2.7,Да,Сакко,Для пиццы,Соль,Pretto,1200,1200,850,...,NaN,1020.0,NaN,NaN,120.0,50,30,20,15,10
2,"Моцарелла ""Unagrande"", 45%, 0,12 кг, ф/п (кубики)",2.7,Да,Альче,Для пиццы,Соль,Unagrande,120,100,850,...,NaN,288.0,25.0,50.0,NaN,50,30,20,15,10
3,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",2.7,Да,Альче,Для пиццы,Соль,Unagrande,3000,100,850,...,NaN,360.0,25.0,50.0,NaN,50,30,20,15,10
4,"Моцарелла (палочки), 45%, кг, пл/л",2.7,Да,Альче,Моцарелла,Соль,Эсперсон,1000,15,850,...,NaN,360.0,NaN,NaN,10.0,50,30,20,15,10
5,"Моцарелла без лактозы для сэндвичей ""Unagrande...",2.7,Нет,Альче,Для пиццы,Соль,Unagrande,280,280,850,...,NaN,1020.0,NaN,NaN,120.0,50,30,20,15,10
6,"Моцарелла в воде Фиор Ди Латте ""Orecchio Oro"",...",3.3,Да,Сакко,Фиор Ди Латте,Вода,Orecchio Oro,100,100,1000,...,NaN,960.0,25.0,50.0,NaN,40,30,35,15,10
7,"Моцарелла в воде Фиор Ди Латте без лактозы ""Un...",3.3,Нет,Альче,Фиор Ди Латте,Вода,Unagrande,125,125,1000,...,NaN,1200.0,25.0,50.0,NaN,40,35,35,15,10
8,"Моцарелла в воде Фиор Ди Латте без лактозы ""Вк...",3.3,Нет,Альче,Фиор Ди Латте,Вода,ВкусВилл,125,125,1000,...,NaN,1200.0,25.0,50.0,NaN,40,35,35,15,10
9,"Моцарелла в воде Чильеджина ""Orecchio Oro"", 45...",3.3,Да,Сакко,Чильеджина,Вода,Orecchio Oro,100,8,1000,...,NaN,1020.0,25.0,50.0,NaN,40,30,35,15,10


In [47]:
boiling_technologies_columns = ['Время налива', 'Время отвердевания', 'Время нарезки', 'Время слива', 'Дополнительное время']
btt_data = tmp[boiling_technologies_columns]
# btt_data.drop_duplicates(keep=False)

In [49]:
btt_data.drop_duplicates()

,Время налива,Время отвердевания,Время нарезки,Время слива,Дополнительное время
0,50,30,20,15,10
6,40,30,35,15,10
7,40,35,35,15,10


In [50]:
 ['Тип закваски', 'Процент', 'Наличие лактозы', 'Линия'] + boiling_technologies_columns

['Тип закваски',
 'Процент',
 'Наличие лактозы',
 'Линия',
 'Время налива',
 'Время отвердевания',
 'Время нарезки',
 'Время слива',
 'Дополнительное время']